<a href="https://colab.research.google.com/github/kento-koyama/bayesian_predictive_micro_ICPMF12/blob/main/Example_1/censored_RJAGS_examp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table class="tfo-notebook-buttons" align="left">
<td>
<a target="_blank" href="https://github.com/kento-koyama/bayesian_predictive_micro_ICPMF12/"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</td>
</table>

# Setting for RJAGS

In [ ]:
uname <- system("uname -a", intern = TRUE)
cat("System information:", uname, "\n")
# Get the current working directory
cwd <- getwd()
cat("Current working directory:", cwd, "\n")
system("sudo apt-get update")
system("sudo apt-get install jags")
# Check the JAGS location
jags_location <- system("which jags", intern = TRUE)
cat("JAGS Location:", jags_location, "\n")
temp_file <- system("mktemp", intern = TRUE)
# Download the JAGS Wiener module .deb file
jags_url <- "https://launchpad.net/~cidlab/+archive/ubuntu/jwm/+files/jags-wiener-module_1.1-5_amd64.deb"
wget_result <- system2("wget", c("-O", temp_file, jags_url), stdout = TRUE, stderr = TRUE)
wget_result
# Install the JAGS Wiener module .deb file
install_result <- system2("sudo", c("dpkg", "-i", temp_file), stdout = TRUE, stderr = TRUE)
# Remove the JAGS Wiener module .deb file
system(paste("rm -f", temp_file))
uname <- system("jags", intern = TRUE)
cat("System information:", uname, "\n")

System information: Linux 5172a14a1ded 5.15.107+ #1 SMP Sat Apr 29 09:15:28 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux 
Current working directory: /content 
JAGS Location: /usr/bin/jags 


[1] "--2023-05-23 14:42:57--  https://launchpad.net/~cidlab/+archive/ubuntu/jwm/+files/jags-wiener-module_1.1-5_amd64.deb"       
 [2] "Resolving launchpad.net (launchpad.net)... 185.125.189.223, 185.125.189.222, 2620:2d:4000:1001::8003, ..."                  
 [3] "Connecting to launchpad.net (launchpad.net)|185.125.189.223|:443... connected."                                             
 [4] "HTTP request sent, awaiting response... 303 See Other"                                                                      
 [5] "Location: https://launchpadlibrarian.net/289923160/jags-wiener-module_1.1-5_amd64.deb [following]"                          
 [6] "--2023-05-23 14:42:57--  https://launchpadlibrarian.net/289923160/jags-wiener-module_1.1-5_amd64.deb"                       
 [7] "Resolving launchpadlibrarian.net (launchpadlibrarian.net)... 185.125.189.228, 185.125.189.229, 2620:2d:4000:1001::8008, ..."
 [8] "Connecting to launchpadlibrarian.net (launchpadlibrarian.net)|185.125.189.228|:443... connected."                           
 [9] "HTTP request sent, awaiting response... 200 OK"                                                                             
[10] "Length: 18376 (18K) [application/x-debian-package]"                                                                         
[11] "Saving to: ‘/tmp/tmp.QL3NsVaSsZ’"                                                                                           
[12] ""                                                                                                                           
[13] "     0K .......... .......                                    100% 95.5M=0s"                                                
[14] ""                                                                                                                           
[15] "2023-05-23 14:42:58 (95.5 MB/s) - ‘/tmp/tmp.QL3NsVaSsZ’ saved [18376/18376]"                                                
[16] ""

System information: Welcome to JAGS 4.3.2 on Tue May 23 14:42:58 2023 JAGS is free software and comes with ABSOLUTELY NO WARRANTY Loading module: basemod: ok Loading module: bugs: ok .  


In [ ]:
#Install and library
install.packages(c("rjags","runjags"))
library(rjags)
library(runjags)

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘coda’


Loading required package: coda

Linked to JAGS 4.3.2

Loaded modules: basemod,bugs



# Bayesian modelling

In [ ]:
#Anne Thebault code inspired from the source
#https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-021-04496-8
library (rjags)
rm(list = ls())
set.seed(1234)

N<-50
y<-rnorm(N,10,3)
# 4 censored values
Ncens=4
y<-sort(y)
yinit<-y
mean(y)
sd(y)
#LOQ 7
censorLimitm = c(rep(y[Ncens],4))
isobserved = c(rep(FALSE,Ncens))
isobserved = as.numeric(isobserved)
y[1:4] = NA


##2 datasets
No=Ncens+1

dataList = list( y = y , No = No,Ncens=Ncens, N=N,
                 isobserved = as.numeric(isobserved) # JAGS dinterval needs 0,1 not T,F
                 , censorLimitm = censorLimitm # vector of limits
                 )



modelstring = "
model {
  for ( i in 1:Ncens ) {
    isobserved[i] ~ dinterval( y[i] , censorLimitm[i] )
    y[i] ~ dnorm( mu , tau ) 
  }
for ( j in No:N ) {
     y[j] ~ dnorm( mu , tau ) 
  }
##Priors
  tau <- 1/pow(sigma,2)
  sigma ~ dunif(0,100)
  mu ~ dnorm(0,1E-6)
}
" # close quote for modelstring
writeLines(modelstring,con="model.txt")
# INTIALIZE THE CHAINS.

sigmaInit = 15
muInit = 100
# intial values of censored data:
yInit = rep( NA , length(y) )
yInit[1:4] = censorLimitm[1:4]


initsList = list( mu=muInit , sigma = sigmaInit, y=yInit)



parameters = c( "mu" , "sigma","y" )
adaptSteps = 1000 # overkill, just shows general form
burnInSteps = 2000 # overkill, just shows general form
nChains = 3 
numSavedSteps=50000
thinSteps=1
nIter = ceiling( ( numSavedSteps * thinSteps ) / nChains )
# Create, initialize, and adapt the model:
jagsModel = jags.model( "model.txt" , data=dataList , inits=initsList , 
                        n.chains=nChains , n.adapt=adaptSteps )
# Burn-in:
cat( "Burning in the MCMC chain...\n" )
update( jagsModel , n.iter=burnInSteps )
# The saved MCMC chain:
cat( "Sampling final MCMC chain...\n" )
codaSamples = coda.samples( jagsModel , variable.names=parameters , 
                            n.iter=nIter , thin=thinSteps )
# resulting codaSamples object has these indices: 
#   codaSamples[[ chainIdx ]][ stepIdx , paramIdx ]
summary(codaSamples)
mu<-rbind(codaSamples[[1]][,1],codaSamples[[2]][,1],codaSamples[[3]][,1])
mean(mu)
parameters1 = c( "mu" , "sigma")
codaSamples = coda.samples( jagsModel , variable.names=parameters1 , 
                            n.iter=nIter , thin=thinSteps )

gelman.diag(codaSamples)
mean(yinit)


[1] 8.640841

[1] 2.655131

Compiling model graph
   Resolving undeclared variables
   Allocating nodes
Graph information:
   Observed stochastic nodes: 50
   Unobserved stochastic nodes: 6
   Total graph size: 70

Initializing model

Burning in the MCMC chain...
Sampling final MCMC chain...



Iterations = 3001:19667
Thinning interval = 1 
Number of chains = 3 
Sample size per chain = 16667 

1. Empirical mean and standard deviation for each variable,
   plus standard error of the mean:

        Mean     SD Naive SE Time-series SE
mu     8.641 0.3917 0.001752       0.001785
sigma  2.727 0.3019 0.001350       0.002028
y[1]   4.268 1.2173 0.005444       0.005806
y[2]   4.275 1.2206 0.005459       0.005796
y[3]   4.272 1.2163 0.005439       0.005680
y[4]   4.277 1.2155 0.005436       0.005754
y[5]   5.977 0.0000 0.000000       0.000000
y[6]   6.244 0.0000 0.000000       0.000000
y[7]   6.379 0.0000 0.000000       0.000000
y[8]   6.497 0.0000 0.000000       0.000000
y[9]   6.678 0.0000 0.000000       0.000000
y[10]  6.794 0.0000 0.000000       0.000000
y[11]  6.929 0.0000 0.000000       0.000000
y[12]  7.005 0.0000 0.000000       0.000000
y[13]  7.017 0.0000 0.000000       0.000000
y[14]  7.094 0.0000 0.000000       0.000000
y[15]  7.192 0.0000 0.000000       0.000000
y[16]  7.

[1] 8.64068

Potential scale reduction factors:

      Point est. Upper C.I.
mu             1          1
sigma          1          1

Multivariate psrf

1

[1] 8.640841